## Leish_Aneuploidy_Multiomics

In [37]:
import os
from DataSelect import *
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

ANALYSISFOLDER= "/Data"

In [38]:
#Load 4'omics data
os.chdir(ANALYSISFOLDER)
    
CNVfile = 'CNVs.txt'
Somyfile= 'Ploidy.txt'
Transcriptfile = 'RNA.txt'
Proteinfile = 'Proteins.txt'
snpdf = pd.read_csv('Ldonovani.vcf', index_col = 0, skiprows = 62, sep ="\t")

anno = pd.read_csv('fullanno.csv', index_col = 0, sep = ";")

snpdf["SNP_type"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[1])
snpdf["SNP_impact"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[2])
snpdf["Genename"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[3])
snpdf["SNP_substitution_base"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[9])
snpdf["SNP_substitution_amino"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[10])
snpdf["SNP_pos_transcript"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[11])
snpdf["SNP_pos_protein"] = snpdf["INFO"].str.split('ANN=').map(lambda x: x[1]).str.split('|').map(lambda x: x[13])

transdict, samples = readomic(Transcriptfile)
proteindict, samples = readomic(Proteinfile)
ploidydict, samples = readomic(Somyfile)
CNVdict, samples = readomic(CNVfile)

In [39]:
somy = pd.DataFrame.from_dict(ploidydict, orient='index', dtype = float)
CNV = pd.DataFrame.from_dict(CNVdict, orient='index', dtype = float)
CNV["chrom"] = ['Ld' + x.split('_')[1][0:2] for x in list(CNV.index)]
CNV2 = CNV.copy()
somypergene = CNV.copy()

def calculate_somy(row, somy, colname):
    return row[colname] * somy[colname][row["chrom"]]

for colname in CNV2:
    if colname != 'chrom':
        if '.1' in colname:
            for index, row in CNV2.iterrows():
                CNV2[colname][index] = calculate_somy(row, somy, colname)

for colname in somypergene:
    if colname != 'chrom':
        if '.1' in colname:
            for index, row in somypergene.iterrows():
                chrom = somypergene['chrom'][index]
                somypergene[colname][index] = somy[colname][chrom]

/tmp/ipykernel_853/2297374771.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CNV2[colname][index] = calculate_somy(row, somy, colname)
/tmp/ipykernel_853/2297374771.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  somypergene[colname][index] = somy[colname][chrom]


<h3> Define Samples </h3>

    Define samples in first and second condition. Samples for transcript and proteins are typically the same, exept for BPK275, where one transcript replicate (275.4) was dropped for sequencing quality reasons.

In [41]:
strains = {'275.1' : 'BPK275A1', '173.1' : 'BPK173A1', '282.1': 'BPK282B2', '575.1' : 'BHU575A1','288.1' : 'BPK288B2', '178.1' : 'BPK178A1'}

combos = [
    {"STRAIN_A" : ['173.1','173.2','173.3','173.4','173L.1', '173L.2'],
    "STRAIN_B" : ['275.1','275.2','275.3','275L.1','275L.2'],
    "STRAIN_B_proteinlevel" : ['275.1','275.2','275.3','275.4','275L.1','275L.2'],
    "outfileID" : 'F_275vs173'},

{"STRAIN_A" : ['288.1','288.2','288.3','288.4'],
"STRAIN_B" : ['275.1','275.2','275.3'],
"STRAIN_B_proteinlevel" : ['275.1','275.2','275.3','275.4'],
"outfileID" : 'F_275vs288'},

{"STRAIN_A" : ['288.1','288.2','288.3','288.4'],
"STRAIN_B" : ['173.1','173.2','173.3','173.4'],
"STRAIN_B_proteinlevel" : ['173.1','173.2','173.3','173.4'],
"outfileID" : 'F_173vs288'},

#
    
{"STRAIN_A" : ['575.1','575.2','575.3','575.4'],
"STRAIN_B" : ['288.1','288.2','288.3','288.4'],
"STRAIN_B_proteinlevel" : ['288.1','288.2','288.3','288.4'],
"outfileID" : 'F_288vs575'},
    
{"STRAIN_A" : ['575.1','575.2','575.3','575.4'],
"STRAIN_B" : ['275.1','275.2','275.3'],
"STRAIN_B_proteinlevel" : ['275.1','275.2','275.3','275.4'],
"outfileID" : 'F_275vs575'},
    
{"STRAIN_A" : ['575.1','575.2','575.3','575.4'],
"STRAIN_B" : ['173.1','173.2','173.3','173.4'],
"STRAIN_B_proteinlevel" : ['173.1','173.2','173.3','173.4'],
"outfileID" : 'F_173vs575'},

#

{"STRAIN_A" : ['282.1','282.2','282.3','282.4'],
"STRAIN_B" : ['275.1','275.2','275.3'],
"STRAIN_B_proteinlevel" : ['275.1','275.2','275.3','275.4'],
"outfileID" : 'F_275vs282'},

{"STRAIN_A" : ['282.1','282.2','282.3','282.4'],
"STRAIN_B" : ['575.1','575.2','575.3','575.4'],
"STRAIN_B_proteinlevel" : ['575.1','575.2','575.3','575.4'],
"outfileID" : 'F_575vs282'},

{"STRAIN_A" : ['282.1','282.2','282.3','282.4'],
"STRAIN_B" : ['288.1','288.2','288.3','288.4'],
"STRAIN_B_proteinlevel" : ['288.1','288.2','288.3','288.4'],
"outfileID" : 'F_288vs282'},
    
{"STRAIN_A" : ['282.1','282.2','282.3','282.4'],
"STRAIN_B" : ['173.1','173.2','173.3','173.4'],
"STRAIN_B_proteinlevel" : ['173.1','173.2','173.3','173.4'],
"outfileID" : 'F_173vs282'},

####################################################

{"STRAIN_A" : ['178.1','178.2','178.3','178.4','178L.1', '178L.2'],
"STRAIN_B" : ['275.1','275.2','275.3','275L.1','275L.2'],
"STRAIN_B_proteinlevel" : ['275.1','275.2','275.3','275.4','275L.1','275L.2'],
"outfileID" : 'F_275vs178'},
    
{"STRAIN_A" : ['178.1','178.2','178.3','178.4'],
"STRAIN_B" : ['288.1','288.2','288.3','288.4'],
"STRAIN_B_proteinlevel" : ['288.1','288.2','288.3','288.4'],
"outfileID" : 'F_288vs178'},
    
{"STRAIN_A" : ['178.1','178.2','178.3','178.4'],
"STRAIN_B" : ['282.1','282.2','282.3','282.4'],
"STRAIN_B_proteinlevel" : ['178.1','178.2','178.3','178.4'],
"outfileID" : 'F_178vs282'},

{"STRAIN_A" : ['178.1','178.2','178.3','178.4'],
"STRAIN_B" : ['575.1','575.2','575.3','575.4'],
"STRAIN_B_proteinlevel" : ['575.1','575.2','575.3','575.4'],
"outfileID" : 'F_575vs178'},

{"STRAIN_A" : ['178.1','178.2','178.3','178.4','178L.1', '178L.2'],
"STRAIN_B" : ['173.1','173.2','173.3','173.4','173L.1', '173L.2'],
"STRAIN_B_proteinlevel" : ['173.1','173.2','173.3','173.4','173L.1', '173L.2'],
"outfileID" : 'F_173vs178'}
]

In [ ]:
summarydict = {}
transcriptpds = []
proteinpds = []
FDR = 0.05

for i in combos:
    strain_A = i["STRAIN_A"][0]
    strain_B = i["STRAIN_B"][0]
    strain_A_replicates = i["STRAIN_A"]
    strain_B_replicates = i["STRAIN_B"]
    outfileID = i["outfileID"]
    strain_B_protein = i["STRAIN_B_proteinlevel"]
    print(outfileID)

    ###
    ###GENOME
    ###

    Dosagetotal = (CNV2[strain_B]/CNV2[strain_A]).to_frame()
    Dosagetotal = np.log2(Dosagetotal)
    Dosagetotal.index.name = 'GeneName'
    Dosagetotal.columns = ['Dosage_Log2FC']

    CNVdif = (CNV[strain_B]/CNV[strain_A]).to_frame()
    CNVdif = np.log2(CNVdif)
    CNVdif.index.name = 'GeneName'
    CNVdif.columns = ['CNV_Log2FC']

    somypergenedif = (somypergene[strain_B]/somypergene[strain_A]).to_frame()
    somypergenedif = np.log2(somypergenedif)
    somypergenedif.index.name = 'GeneName'
    somypergenedif.columns = ['Somy_Log2FC']

    ###
    ###TRANSCRIPTOME
    ###
    
    ##
    ##Normalisation
    ##
    
    log2FC_Trans = pd.DataFrame()
    df = pd.DataFrame.from_dict(transdict, orient='index', dtype = float)
    df = df.drop(['275.4'], axis=1)
    #add pseudocounts for remaining zeros
    df = df.replace(0, 1)
    #take 2 log
    df = np.log2(df)
    #divide column by column mean
    for column in df:
        df[column] = df[column] - np.average(df[column])
       
    ##
    ##Calculate Fold Change
    ##
    
    denominator = df[strain_A_replicates]
    numerator = df[strain_B_replicates]
    log2FC = numerator.mean(axis = 1) - denominator.mean(axis = 1)
    log2FC = log2FC.to_frame('transcript_log2fc')
    for index, row in numerator.iterrows():
        log2FC.loc[index,'transcript_pval'] = ttest_ind(numerator.loc[index], denominator.loc[index])[1]
    log2FC_Trans = log2FC_Trans.append(log2FC)
    log2FC_Trans.index.name ='GeneName'
    log2FC_Trans["GeneName2"] = log2FC_Trans.index

    ###
    ###PROTEOME
    ###
    
    ##
    ##Normalisation
    ##

    df = pd.DataFrame.from_dict(proteindict, orient='index', dtype = float)
    df = df.replace(0, np.nan)
    df.describe()
    normprot = df
    
    ##
    ##Calculate Fold Change
    ##
    
    log2FC_Prot = pd.DataFrame()
    denominator = np.log2(normprot[strain_A_replicates])
    numerator = np.log2(normprot[strain_B_protein])
    log2FC = numerator.mean(axis = 1) - denominator.mean(axis = 1)
    log2FC = log2FC.to_frame('protein_log2fc')
    for index, row in numerator.iterrows():
        log2FC.loc[index,'pval_protein'] = ttest_ind(numerator.loc[index].dropna(), denominator.loc[index].dropna())[1]
        log2FC.loc[index,'samplesizeSample_protein'] = len(numerator.loc[index].dropna())
        log2FC.loc[index,'samplesizeControl_protein'] = len(denominator.loc[index].dropna())
        log2FC.loc[index,'nstdev_sample'] = numerator.loc[index].dropna().std()/numerator.loc[index].dropna().mean()
        log2FC.loc[index,'nstdev_control'] = denominator.loc[index].dropna().std()/denominator.loc[index].dropna().mean()
    log2FC_Prot = log2FC_Prot.append(log2FC)
    log2FC_Prot.index.name ='GeneName'
    
    ###
    ###INTEGRATE
    ###
    
    ##
    ## Integrate
    ##
    
    result = pd.merge(log2FC_Trans, log2FC_Prot, left_on='GeneName', right_on='GeneName',sort=True, how='left')
    result = pd.merge(result, Dosagetotal, left_on='GeneName', right_on='GeneName',sort=True, how='left')
    result = pd.merge(result, CNVdif, left_on='GeneName', right_on='GeneName',sort=True, how='left')
    result = pd.merge(result, somypergenedif, left_on='GeneName', right_on='GeneName',sort=True, how='left')
    result["compensation"] = result["protein_log2fc"] - result["transcript_log2fc"]
    result["Strains"] = outfileID[2:]
    result = result[(result ['CNV_Log2FC'] < 0.3) & (result ['CNV_Log2FC'] > -0.3)]

    anno['LdBPKV2'] = anno['LdBPKV2'].str.replace('.1','', regex = False)

    result = pd.merge(result, anno, left_on = "GeneName", right_on='LdBPKV2',sort=True, how='left').drop(columns=["Eccentricity", "IsSingleNode",
                                'ClosenessCentrality','ClusteringCoefficient',"Stress","Degree",
                                "Radiality","BetweennessCentrality","PartnerOfMultiEdgedNodePairs","NeighborhoodConnectivity","Gravy",
                                 "NumberOfDirectedEdges","NumberOfUndirectedEdges","AverageShortestPathLength","TopologicalCoefficient","degree"])
    result = result[~result.LdBPKV2.str.contains("LdBPK_3700", na=False)]
    result['SNPs'] = result.apply(lambda row: checkSNP(strains[strain_B],strains[strain_A], row["L. donovani"], snpdf), axis=1)
    proteinpds.append(result)
    
    ##
    ## Calculate cis- en trans-transcripts
    ##
    log2FC_Trans_sel = result[result['transcript_pval'].notna()]
    log2FC_Trans_sel['trans_padj'] = multipletests(log2FC_Trans_sel['transcript_pval'], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    log2FC_Trans_sel["aneuploidy-driven"] = (log2FC_Trans_sel['trans_padj'] < FDR) & (log2FC_Trans_sel['Somy_Log2FC'].abs() < 0.30)
    cis = len(log2FC_Trans_sel[(log2FC_Trans_sel['trans_padj'] < FDR) & (log2FC_Trans_sel['Somy_Log2FC'].abs() > 0.30)])
    trans = len(log2FC_Trans_sel[(log2FC_Trans_sel['trans_padj'] < FDR) & (log2FC_Trans_sel['Somy_Log2FC'].abs() < 0.30)])
    total = float(cis) + float(trans)
    if total != 0:
        summarydict[outfileID[2:]] = {"protein" :0, "transcript" : [cis,trans,total,(float(cis)/total*100.0), (float(trans)/total*100.0)]}
    else:
        summarydict[outfileID[2:]] = {"protein" : 0, "transcript" : [cis,trans,total,"NA","NA"]}
    log2FC_Trans_sel.index.name ='GeneName'
    transcriptpds.append(log2FC_Trans_sel)

In [43]:
pd.concat(proteinpds, axis=0).to_csv('allcomp_final.csv', sep = '\t')

In [52]:
FDR = 0.05
samplesize = 3
FC = 0.0

Differential_Proteins = []
totaltrans = {}
totalprot = {}

dfObj = pd.DataFrame(columns=['Comparisson', 'Diff_Total','Diff_Cis','Diff_Trans','Direction', 'Proteins_All','Proteins_Cis','Proteins_Trans'])

totaldif = 0
newpds = []
for i in range(len(proteinpds)):
    strains = proteinpds[i].loc[1,"Strains"]
    log2FC_Prot_sel = proteinpds[i]
    log2FC_Prot_sel = log2FC_Prot_sel[log2FC_Prot_sel['samplesizeSample_protein'] > samplesize]
    log2FC_Prot_sel = log2FC_Prot_sel[log2FC_Prot_sel['samplesizeControl_protein'] > samplesize]
    log2FC_Prot_sel = log2FC_Prot_sel[log2FC_Prot_sel['pval_protein'].notna()]
    log2FC_Prot_sel['prot_padj'] = multipletests(log2FC_Prot_sel['pval_protein'], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    log2FC_Prot_sel["aneuploidy-driven"] = (log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() < 0.30)
    log2FC_Prot_sel["L. major"] =  log2FC_Prot_sel["L. major"].str.split(',').str[0]
    newpds.append(log2FC_Prot_sel)
    
# Need to make sure it matches the right L.donovani gene
    difprot = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel["protein_log2fc"] > 0)]["L. major"].to_list()
    up = len(difprot)
    totaldif += len(difprot)
    upproteins = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel["protein_log2fc"] > 0)]["L. major"].dropna().to_list()
    difprot = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel["protein_log2fc"] < 0)]["L. major"].to_list()
    down = len(difprot)
    totaldif += len(difprot)
    downproteins = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel["protein_log2fc"] < 0)]["L. major"].dropna().to_list()
    
    ##
    ## Calculate cis- en trans-proteins
    ##
    
    cis = len(log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() >= 0.30)])
    trans = len(log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() < 0.30)])
    total = float(cis) + float(trans)
    if total != 0:
        summarydict[strains]['protein'] = [cis,trans,total,(float(cis)/total*100.0), (float(trans)/total*100.0)]
    else:
        summarydict[strains]['protein'] = [cis,trans,total,"NA","NA"]
        
    ##
    ## Determine cis- en trans-proteins up-and down
    ##
    
    log2FC_Prot_sel = log2FC_Prot_sel.sort_values(by=['Somy_Log2FC'], ascending=False)
    for prot in log2FC_Prot_sel["L. major"].dropna().to_list():
        totalprot[prot] = 0

    cisup = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() >= 0.30) & (log2FC_Prot_sel["protein_log2fc"] >= FC)]
    transup = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() < 0.30) & (log2FC_Prot_sel["protein_log2fc"] >= FC)]
    cisdown = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() >= 0.30) & (log2FC_Prot_sel["protein_log2fc"] < -FC)]
    transdown = log2FC_Prot_sel[(log2FC_Prot_sel['prot_padj'] < FDR) & (log2FC_Prot_sel['Somy_Log2FC'].abs() < 0.30) & (log2FC_Prot_sel["protein_log2fc"] < -FC)]
    allup = pd.concat([cisup, transup], ignore_index=True)
    alldown = pd.concat([cisdown, transdown], ignore_index=True)
    alltrans = pd.concat([transup, transdown], ignore_index=True)
    
    for prot in alltrans["L. major"].dropna().to_list():
        totaltrans[prot] = 0
    
    cisup_lmaj = cisup["L. major"].dropna().to_list()
    transup_lmaj = transup["L. major"].dropna().to_list()
    cisdown_lmaj = cisdown["L. major"].dropna().to_list()
    transdown_lmaj = transdown["L. major"].dropna().to_list()
    allup_lmaj = pd.concat([cisup, transup], ignore_index=True)["L. major"].dropna().to_list()
    alldown_lmaj = pd.concat([cisdown, transdown], ignore_index=True)["L. major"].dropna().to_list()
    
    dfObj = dfObj.append({'Comparisson' : strains, 'Diff_Total' : len(allup_lmaj), 'Diff_Cis' : len(cisup_lmaj), 'Diff_Trans' : len(transup_lmaj) , 'Direction' : 'Up', 'Proteins_All' : " ".join([i.split(',')[0] for i in allup_lmaj]),'Proteins_Cis' : " ".join([i.split(',')[0] for i in cisup_lmaj]), 'Proteins_Trans' : " ".join([i.split(',')[0] for i in transup_lmaj])}, ignore_index=True)
    dfObj = dfObj.append({'Comparisson' : strains, 'Diff_Total' : len(alldown_lmaj), 'Diff_Cis' : len(cisdown_lmaj), 'Diff_Trans' : len(transdown_lmaj), 'Direction' : 'Down', 'Proteins_All' : " ".join([i.split(',')[0] for i in alldown_lmaj]),'Proteins_Cis' : " ".join([i.split(',')[0] for i in cisdown_lmaj]), 'Proteins_Trans' : " ".join([i.split(',')[0] for i in transdown_lmaj])}, ignore_index=True)
    
    cisup['Cis/Trans'] = "Cis"
    cisdown['Cis/Trans'] = "Cis"
    transup['Cis/Trans'] = "Trans"
    transdown['Cis/Trans'] = "Trans"
    Differential_Proteins.append(pd.concat([cisup,cisdown,transup,transdown]))

/tmp/ipykernel_853/3562603053.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cisup['Cis/Trans'] = "Cis"
/tmp/ipykernel_853/3562603053.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cisdown['Cis/Trans'] = "Cis"
/tmp/ipykernel_853/3562603053.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

/tmp/ipykernel_853/3562603053.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cisup['Cis/Trans'] = "Cis"
/tmp/ipykernel_853/3562603053.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cisdown['Cis/Trans'] = "Cis"
/tmp/ipykernel_853/3562603053.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [45]:
pd.concat(Differential_Proteins).to_csv('CisTrans_final.csv', sep = ';')

## Table S1.H Trans-Proteins

In [46]:
df = pd.concat(Differential_Proteins)
df = df[df["Cis/Trans"] == "Trans"]
df.to_csv('test.csv', sep = ";")
df['Duplicated'] = df['GeneName2'].duplicated(keep = False)
df['In_Comparissons'] = df.groupby('GeneName2')['Strains'].transform(lambda x : ', '.join(x))
df = df[["GeneName2","GeneID","Duplicated","In_Comparissons","SNPs", "anno"]]
df = df.set_index("GeneName2")
df['SNPs'] = df['SNPs'].astype(str)
df = df.drop_duplicates()
df.to_csv('Trans_Proteins_final.csv', sep = ';')
df.shape

(203, 5)

## Table S1.G Trans-Transcripts

In [47]:
df = pd.concat(transcriptpds)
df = df[df["aneuploidy-driven"] == False]
df = df[df["trans_padj"] < FDR]
df['Duplicated'] = df['GeneName2'].duplicated(keep = False)
df['In_Comparissons'] = df.groupby('GeneName2')['Strains'].transform(lambda x : ', '.join(x))
df = df[["GeneName2","GeneID","Duplicated","In_Comparissons","SNPs", "anno"]]
df = df.set_index("GeneName2")
df['SNPs'] = df['SNPs'].astype(str)
df = df.drop_duplicates()
df.to_csv('Trans_Transcripts_final.csv', sep = ';')
df.shape

(541, 5)

## Table S1.F Differential Transcripts & Proteins

In [48]:
for i in summarydict:
    print("\t".join([str(j) for j in [i, summarydict[i]['transcript'][3],summarydict[i]['transcript'][4],summarydict[i]['transcript'][2],summarydict[i]['protein'][3],summarydict[i]['protein'][4],summarydict[i]['protein'][2]]]))

275vs173	86.56716417910447	13.432835820895523	134.0	40.65934065934066	59.34065934065934	91.0
275vs288	90.9090909090909	9.090909090909092	11.0	48.0	52.0	125.0
173vs288	86.25	13.750000000000002	80.0	55.55555555555556	44.44444444444444	27.0
288vs575	65.75342465753424	34.24657534246575	73.0	71.42857142857143	28.57142857142857	14.0
275vs575	84.82758620689656	15.172413793103448	145.0	58.92857142857143	41.07142857142857	56.0
173vs575	NA	NA	0.0	50.0	50.0	4.0
275vs282	77.61194029850746	22.388059701492537	67.0	46.42857142857143	53.57142857142857	84.0
575vs282	41.17647058823529	58.82352941176471	34.0	6.25	93.75	16.0
288vs282	60.86956521739131	39.130434782608695	23.0	39.34426229508197	60.65573770491803	61.0
173vs282	74.35897435897436	25.64102564102564	39.0	48.148148148148145	51.85185185185185	27.0
275vs178	79.79274611398964	20.207253886010363	386.0	44.761904761904766	55.23809523809524	105.0
288vs178	54.54545454545454	45.45454545454545	11.0	45.45454545454545	54.54545454545454	11.0
178vs282	71.42857